In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import random
import os
import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [12]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
torch.manual_seed(SEED)
torch.use_deterministic_algorithms(True)

#### The Data

In [13]:
# df = pd.read_csv('../../data/more_features/more_feats_correlated_testing_noanem_noisy_6.csv')
# df = df.fillna(-1)
# df.head()
train_df = pd.read_csv('../../final/data/train_set_noisy_4.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,11.786358,21.598977,2.609478,4.819682,-1.000000,75.211769,242.694150,4.701269,0,1.491966,135.606446,123.227658,59.636941,0.56255,69.236844,35.359073,-1.00000,4.0
1,9.765445,95.948185,-1.000000,-1.000000,473.810060,79.529629,-1.000000,3.683701,1,0.756766,86.763989,-1.000000,-1.000000,-1.00000,-1.000000,29.296335,-1.00000,4.0
2,10.581471,264.299544,4.479510,0.401345,232.378777,86.962200,207.714187,3.650369,1,0.887986,30.558329,-1.000000,9.212390,-1.00000,122.595648,31.744414,89.38604,5.0
3,8.455727,339.601244,5.068134,4.201679,408.449185,78.280967,-1.000000,3.240530,0,1.273017,29.899966,-1.000000,-1.000000,-1.00000,-1.000000,25.367180,-1.00000,3.0
4,7.347738,61.963008,-1.000000,-1.000000,472.934836,78.060937,-1.000000,2.823847,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,22.043215,-1.00000,4.0


In [14]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [15]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

In [16]:
action_list = list(constants.CLASS_DICT.keys()) + [col  for col in train_df.columns if col!='label']
action_list

['No anemia',
 'Vitamin B12/Folate deficiency anemia',
 'Unspecified anemia',
 'Anemia of chronic disease',
 'Iron deficiency anemia',
 'Hemolytic anemia',
 'Aplastic anemia',
 'Inconclusive diagnosis',
 'hemoglobin',
 'ferritin',
 'ret_count',
 'segmented_neutrophils',
 'tibc',
 'mcv',
 'serum_iron',
 'rbc',
 'gender',
 'creatinine',
 'cholestrol',
 'copper',
 'ethanol',
 'folate',
 'glucose',
 'hematocrit',
 'tsat']

#### Training 

In [ ]:
for steps in [int(9e6), int(9.5e6), int(10e6), int(10.5e6), int(11e6), int(12e6), int(13e6)]:
    dqn_model = utils.stable_dqn3(X_train, y_train, steps, True, f'../../final/models/dqn_noisy_4_{steps}')

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.11     |
|    ep_rew_mean      | -0.86    |
|    exploration_rate | 0.69     |
|    success_rate     | 0.09     |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 844      |
|    time_elapsed     | 348      |
|    total_timesteps  | 293902   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 7.77     |
|    n_updates        | 60975    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.88     |
|    ep_rew_mean      | -0.9     |
|    exploration_rate | 0.322    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 735      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.68     |
|    ep_rew_mean      | -0.02    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.51     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 584      |
|    time_elapsed     | 11689    |
|    total_timesteps  | 6829443  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.098    |
|    n_updates        | 1694860  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.88     |
|    ep_rew_mean      | -0.16    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.5      |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 572      |
|    time_elapsed     | 12750    |
|    total_timesteps  | 7296547  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.59     |
|    ep_rew_mean      | -0.1     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.47     |
| time/               |          |
|    episodes         | 1100000  |
|    fps              | 648      |
|    time_elapsed     | 6413     |
|    total_timesteps  | 4159730  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.07     |
|    n_updates        | 1027432  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.35     |
|    ep_rew_mean      | -0.02    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.51     |
| time/               |          |
|    episodes         | 1200000  |
|    fps              | 645      |
|    time_elapsed     | 7016     |
|    total_timesteps  | 4526307  |
| train/              |          |
|    learning_rate  

using stable baselines 3
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.49     |
|    ep_rew_mean      | -0.88    |
|    exploration_rate | 0.722    |
|    success_rate     | 0.1      |
| time/               |          |
|    episodes         | 100000   |
|    fps              | 572      |
|    time_elapsed     | 511      |
|    total_timesteps  | 292542   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 2.81     |
|    n_updates        | 60635    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.69     |
|    ep_rew_mean      | -0.92    |
|    exploration_rate | 0.389    |
|    success_rate     | 0.07     |
| time/               |          |
|    episodes         | 200000   |
|    fps              | 497      |
|    t

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.3      |
|    ep_rew_mean      | 0.08     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.62     |
| time/               |          |
|    episodes         | 1600000  |
|    fps              | 364      |
|    time_elapsed     | 18597    |
|    total_timesteps  | 6772424  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.0507   |
|    n_updates        | 1680605  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.65     |
|    ep_rew_mean      | -0.06    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.56     |
| time/               |          |
|    episodes         | 1700000  |
|    fps              | 373      |
|    time_elapsed     | 19301    |
|    total_timesteps  | 7200334  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.58     |
|    ep_rew_mean      | -0.32    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.41     |
| time/               |          |
|    episodes         | 800000   |
|    fps              | 716      |
|    time_elapsed     | 4286     |
|    total_timesteps  | 3069836  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.125    |
|    n_updates        | 754958   |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.34     |
|    ep_rew_mean      | -0.22    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.42     |
| time/               |          |
|    episodes         | 900000   |
|    fps              | 701      |
|    time_elapsed     | 4983     |
|    total_timesteps  | 3493817  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.38     |
|    ep_rew_mean      | 0.18     |
|    exploration_rate | 0.05     |
|    success_rate     | 0.65     |
| time/               |          |
|    episodes         | 2300000  |
|    fps              | 642      |
|    time_elapsed     | 14713    |
|    total_timesteps  | 9453255  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.156    |
|    n_updates        | 2350813  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.78     |
|    ep_rew_mean      | 0        |
|    exploration_rate | 0.05     |
|    success_rate     | 0.53     |
| time/               |          |
|    episodes         | 2400000  |
|    fps              | 641      |
|    time_elapsed     | 15384    |
|    total_timesteps  | 9861623  |
| train/              |          |
|    learning_rate  

----------------------------------
| rollout/            |          |
|    ep_len_mean      | 4.07     |
|    ep_rew_mean      | -0.14    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.45     |
| time/               |          |
|    episodes         | 1300000  |
|    fps              | 500      |
|    time_elapsed     | 10280    |
|    total_timesteps  | 5148331  |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.133    |
|    n_updates        | 1274582  |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 3.77     |
|    ep_rew_mean      | -0.04    |
|    exploration_rate | 0.05     |
|    success_rate     | 0.54     |
| time/               |          |
|    episodes         | 1400000  |
|    fps              | 502      |
|    time_elapsed     | 11055    |
|    total_timesteps  | 5559205  |
| train/              |          |
|    learning_rate  

In [ ]:
# class_dict = constants.CLASS_DICT
# df['label'] = df['label'].replace(class_dict)
# X = df.iloc[:, 0:-1]
# y = df.iloc[:, -1]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
# X_train, y_train = np.array(X_train), np.array(y_train)
# X_test, y_test = np.array(X_test), np.array(y_test)
# X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# %%time
# timesteps = int(2e6)
# dqn_model = utils.stable_dqn3(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn3_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

#### Testing

In [ ]:
# training_env = utils.create_env(X_train, y_train)
# dqn_model = utils.load_dqn3('../../models/many_features/0.1/with_correlated_fts/dqn3_by_type_noisy_6_16000000', training_env)
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [ ]:
success_rate, success_df = utils.success_rate(test_df)
success_rate

In [ ]:
avg_length, avg_return = utils.get_avg_length_reward(test_df)
avg_length, avg_return

In [ ]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

In [ ]:
test_df.y_pred.unique()

In [ ]:
test_df[test_df.y_pred==4]

#### Saving files

In [ ]:
# test_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/test_df3_9000000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/0.1/correlated/success_df3_9000000.csv', index=False)

#### Confusion matrix and classification report

In [ ]:
test_df = pd.read_csv('../../test_dfs/many_features/0.1/test_df3_6500000.csv')
test_df.head()

In [ ]:
utils.plot_classification_report(test_df['y_actual'], test_df['y_pred'])

In [ ]:
# def plot_confusion_matrix(y_actual, y_pred, save=False, filename=False):
#     from sklearn.metrics import confusion_matrix
#     cm = confusion_matrix(y_actual, y_pred)
#     cm_df = pd.DataFrame(cm, index = [0, 1, 2, 3, 4, 5, 6], columns = [0, 1, 2, 3, 4, 5, 6], dtype='object')
#     #cm_df = pd.DataFrame(cm, index = constants.CLASS_DICT.keys(), columns = constants.CLASS_DICT.keys())
#     plt.figure(figsize=(8, 6))
#     sns.heatmap(cm_df, annot=True)
#     plt.title('Confusion Matrix')
#     plt.ylabel('Actual Anemia')
#     plt.xlabel('Predicted Anemia')
#     plt.tight_layout()
#     if save:
#         plt.savefig(filename)
#     plt.show()
#     plt.close()

In [ ]:
utils.plot_confusion_matrix(test_df['y_actual'], test_df['y_pred'])